In [ ]:
# ==============================================================================
# BƯỚC 1: CÀI ĐẶT MÔI TRƯỜNG & MOUNT DRIVE
# ==============================================================================
import os
import csv
import time
import sys
from google.colab import drive

# 1. Mount Google Drive
print("📂 Đang kết nối Google Drive...")
drive.mount('/content/drive')

# 2. Cài đặt thư viện cần thiết (yt-dlp, faster-whisper)
print("⬇️ Đang cài đặt thư viện (yt-dlp, faster-whisper)...")
!apt-get install ffmpeg -y > /dev/null
!pip install yt-dlp faster-whisper > /dev/null 2>&1

import yt_dlp
from faster_whisper import WhisperModel

# ==============================================================================
# BƯỚC 2: CẤU HÌNH ĐƯỜNG DẪN (QUAN TRỌNG)
# ==============================================================================

# Đường dẫn đến thư mục bạn đã tạo trên Drive
PROJECT_FOLDER = "/content/drive/MyDrive/TikTok_Project"

# Tên các file (Đảm bảo bạn đã upload đúng tên file này lên thư mục trên)
INPUT_CSV = os.path.join(PROJECT_FOLDER, "fake2_2.csv")
COOKIES_FILE = os.path.join(PROJECT_FOLDER, "cookies.txt")

# File kết quả sẽ được tạo tại đây
OUTPUT_CSV = os.path.join(PROJECT_FOLDER, "output_transcript2.csv")

# Kiểm tra file đầu vào
if not os.path.exists(INPUT_CSV):
    print(f"❌ LỖI: Không tìm thấy file '{INPUT_CSV}'")
    print(f"👉 Hãy tạo thư mục 'TikTok_Project' trên Drive và upload file csv vào đó!")
    sys.exit() # Dừng chương trình

# ==============================================================================
# BƯỚC 3: LOAD MODEL WHISPER LÊN GPU
# ==============================================================================
print("🚀 Đang load model Whisper Large-v3 lên GPU T4...")
# Dùng float16 để chạy nhanh hơn trên GPU
model = WhisperModel("large-v3", device="cuda", compute_type="float16")
print("✅ Đã load xong model! Sẵn sàng chiến đấu.")

# ==============================================================================
# BƯỚC 4: CÁC HÀM XỬ LÝ CHÍNH
# ==============================================================================

def download_audio(video_url, video_id):
    """Download audio từ TikTok dùng yt-dlp với Cookies"""
    if not video_url: return None

    # Đường dẫn file tạm (Lưu trên Colab cho nhanh, xong xóa)
    temp_output = f"/content/temp_{video_id}"

    ydl_opts = {
        'outtmpl': f'{temp_output}.%(ext)s',
        'format': 'bestaudio/best', # Chỉ lấy audio cho nhẹ
        'noplaylist': True,
        'quiet': True,
        'no_warnings': True,
        'ignoreerrors': True,
        # Giả danh trình duyệt
        'http_headers': {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Referer': 'https://www.tiktok.com/',
        },
        # Dùng cookies từ Drive để né chặn (QUAN TRỌNG)
        'cookiefile': COOKIES_FILE if os.path.exists(COOKIES_FILE) else None
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([video_url])

        # Tìm file vừa tải (có thể là .m4a, .mp3, .webm...)
        for ext in ['.m4a', '.mp3', '.webm', '.mp4', '.wav']:
            path = f"{temp_output}{ext}"
            if os.path.exists(path):
                return path
        return None
    except Exception as e:
        # print(f"Lỗi tải: {e}")
        return None

def transcribe_with_whisper(audio_path):
    """Chuyển đổi Audio sang Text"""
    try:
        # Beam_size=5 giúp tăng độ chính xác
        segments, info = model.transcribe(audio_path, language="vi", beam_size=5)

        # Gộp các đoạn text lại
        full_text = " ".join([segment.text for segment in segments])
        return full_text.strip()
    except Exception as e:
        print(f"❌ Lỗi Transcribe: {e}")
        return ""

def read_urls_from_csv(file_path):
    urls = []
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                if 'url' in row and row['url']:
                    urls.append(row['url'])
    except Exception as e:
        print(f"Lỗi đọc CSV: {e}")
    return urls

# ==============================================================================
# BƯỚC 5: CHẠY VÒNG LẶP CHÍNH (MAIN LOOP)
# ==============================================================================

def main():
    print(f"\n📂 Input: {INPUT_CSV}")
    print(f"💾 Output: {OUTPUT_CSV}")

    all_urls = read_urls_from_csv(INPUT_CSV)
    print(f"📊 Tổng số video cần xử lý: {len(all_urls)}")

    # --- CƠ CHẾ RESUME (QUAN TRỌNG) ---
    processed_urls = set()
    file_exists = os.path.exists(OUTPUT_CSV)

    if file_exists:
        try:
            with open(OUTPUT_CSV, 'r', encoding='utf-8') as f:
                reader = csv.DictReader(f)
                for row in reader:
                    processed_urls.add(row['url'])
            print(f"🔄 Tìm thấy file cũ. Đã xử lý: {len(processed_urls)} video. Đang bỏ qua...")
        except:
            print("⚠️ File cũ lỗi hoặc trống, sẽ ghi lại từ đầu.")

    # --- BẮT ĐẦU VÒNG LẶP ---
    print("🚀 BẮT ĐẦU CRAWL...\n")

    count = 0
    for i, url in enumerate(all_urls, 1):
        if url in processed_urls:
            continue # Bỏ qua video đã làm rồi

        print(f"[{i}/{len(all_urls)}] ⏳ {url}")

        audio_path = None
        text_result = ""

        try:
            # 1. Download
            unique_id = f"{i}_{int(time.time())}"
            audio_path = download_audio(url, unique_id)

            if audio_path:
                # 2. Transcribe
                text_result = transcribe_with_whisper(audio_path)

                if text_result:
                    print(f"   ✅ Text ({len(text_result)} chars): {text_result[:60]}...")
                else:
                    print("   ⚠️ Video không có lời hoặc lỗi model.")
            else:
                print("   ❌ Không tải được (Link chết hoặc bị chặn chặn)")

            # 3. Lưu ngay lập tức vào Drive
            mode = 'a' if os.path.exists(OUTPUT_CSV) else 'w'
            with open(OUTPUT_CSV, mode, newline='', encoding='utf-8') as f:
                fieldnames = ['url', 'text']
                writer = csv.DictWriter(f, fieldnames=fieldnames)

                if mode == 'w':
                    writer.writeheader()

                writer.writerow({'url': url, 'text': text_result})

            # Đánh dấu là đã làm
            processed_urls.add(url)
            count += 1

            # Flush bộ nhớ đệm xuống đĩa để chắc chắn dữ liệu an toàn
            f.close()

        except Exception as e:
            print(f"   ❌ Lỗi không xác định: {e}")

        finally:
            # 4. Dọn dẹp file tạm trên Colab để không đầy ổ cứng
            if audio_path and os.path.exists(audio_path):
                os.remove(audio_path)

    print(f"\n🎉 HOÀN THÀNH! Đã xử lý thêm {count} video.")
    print(f"💾 File kết quả nằm tại: {OUTPUT_CSV}")

if __name__ == "__main__":
    main()

📂 Đang kết nối Google Drive...
Mounted at /content/drive
⬇️ Đang cài đặt thư viện (yt-dlp, faster-whisper)...
🚀 Đang load model Whisper Large-v3 lên GPU T4...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

vocabulary.json: 0.00B [00:00, ?B/s]

...

   ❌ Không tải được (Link chết hoặc bị chặn chặn)
[1195/1333] ⏳ https://www.tiktok.com/@7290930690635236358/video/7559803737649204487
   ✅ Text (70 chars): Hãy subscribe cho kênh La La School Để không bỏ lỡ những vid...
[1196/1333] ⏳ https://www.tiktok.com/@7216219477485765658/video/7295234291200511233
   ✅ Text (627 chars): Cách để các bạn sáng tạo hơn  Có bao giờ bạn đang đi ẻ xong ...
[1197/1333] ⏳ https://www.tiktok.com/@7074230314642965546/video/7560869664536939831
   ✅ Text (69 chars): Hãy subscribe cho kênh Ghiền Mì Gõ Để không bỏ lỡ những vide...
[1198/1333] ⏳ https://www.tiktok.com/@7257399739325465606/video/7417691042717191442
   ✅ Text (130 chars): Cảm ơn các bạn đã theo dõi và hẹn gặp lại.  Cảm ơn các bạn đ...
[1199/1333] ⏳ https://www.tiktok.com/@7438813404024144913/video/7548287132696055061
   ✅ Text (570 chars): Một số người đã yêu cầu chốt quyền thi đấu của vận động viên...
[1200/1333] ⏳ https://www.tiktok.com/@6720612111628960770/video/7529163700108905736
   ✅ Text (90

ERROR: [TikTok] 7504957707498065160: Your IP address is blocked from accessing this post


   ❌ Không tải được (Link chết hoặc bị chặn chặn)
[1281/1333] ⏳ https://www.tiktok.com/@7222112498009637914/video/7297177392471084306
   ✅ Text (70 chars): Hãy subscribe cho kênh La La School Để không bỏ lỡ những vid...
[1282/1333] ⏳ https://www.tiktok.com/@7379484354513617940/video/7399529504038751495
   ✅ Text (289 chars): Hãy subscribe cho kênh La La School Để không bỏ lỡ những vid...
[1283/1333] ⏳ https://www.tiktok.com/@7177716522785326107/video/7193309578728049947
   ✅ Text (70 chars): Hãy subscribe cho kênh La La School Để không bỏ lỡ những vid...
[1284/1333] ⏳ https://www.tiktok.com/@7510662315714593800/video/7572792611547696392
   ✅ Text (635 chars): Bạn sắp trở nên cực kỳ giàu có, vì vũ trụ đã chọn bạn làm ng...
[1285/1333] ⏳ https://www.tiktok.com/@6591091945711763458/video/7524153563275103495
   ✅ Text (70 chars): Hãy subscribe cho kênh La La School Để không bỏ lỡ những vid...
[1286/1333] ⏳ https://www.tiktok.com/@7123951161100026926/video/7129473489286794542
   ✅ Text (70 

ERROR: [TikTok] 7549403929096899858: Your IP address is blocked from accessing this post


   ❌ Không tải được (Link chết hoặc bị chặn chặn)
[1321/1333] ⏳ https://www.tiktok.com/@7182473181063267355/video/7321935910587649282
   ✅ Text (217 chars): Hãy subscribe cho kênh La La School Để không bỏ lỡ những vid...
[1322/1333] ⏳ https://www.tiktok.com/@6999408815398847514/video/7567895009047956743
   ✅ Text (145 chars): Hãy subscribe cho kênh La La School Để không bỏ lỡ những vid...
[1323/1333] ⏳ https://www.tiktok.com/@7368796064324125714/video/7432656993862585617
   ✅ Text (353 chars): Hãy subscribe cho kênh Ghiền Mì Gõ Để không bỏ lỡ những vide...
[1324/1333] ⏳ https://www.tiktok.com/@7241236314061997062/video/7268592417417317637
   ✅ Text (145 chars): Hãy subscribe cho kênh La La School Để không bỏ lỡ những vid...
[1325/1333] ⏳ https://www.tiktok.com/@7350646637188645890/video/7387674314541124871
   ✅ Text (145 chars): Hãy subscribe cho kênh La La School Để không bỏ lỡ những vid...
[1326/1333] ⏳ https://www.tiktok.com/@6960088182609642497/video/7038774670618856706
   ✅ Text (